In [ ]:
#hide
#from your_lib.core import *

# AutoFE: Deep Learning-based Automatic Feature Extraction 

> Automatic Feature Extraction in Images and Texts using Transfer Learning

## Install

`pip install auto_feature_extraction`